In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

In [3]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
s = StandardScaler()
df = s.fit_transform(df)
test = s.transform(test)

# m = lambda *x: [i.shape for i in x]
# x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
# print m(x_train, y_train, x_test, y_test)



(69713, 22)
(30037, 21)
CPU times: user 56.1 s, sys: 36.4 s, total: 1min 32s
Wall time: 1min 32s


In [4]:
1

1

In [5]:
%%time
d_train = xgb.DMatrix(df, label = y)
# d_test = xgb.DMatrix(x_test, label=y_test)

d_pred = xgb.DMatrix(test)

CPU times: user 47.2 s, sys: 21.7 s, total: 1min 8s
Wall time: 1min 8s


In [6]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.05,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train')]



In [7]:
start=time()

clf = xgb.train(params, d_train, 143, watchlist, early_stopping_rounds=10, 
                verbose_eval=3,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.752004
Will train until train-auc hasn't improved in 10 rounds.
[3]	train-auc:0.834489
[6]	train-auc:0.84568
[9]	train-auc:0.854612
[12]	train-auc:0.863002
[15]	train-auc:0.869551
[18]	train-auc:0.874136
[21]	train-auc:0.883115
[24]	train-auc:0.887652
[27]	train-auc:0.891738
[30]	train-auc:0.896189
[33]	train-auc:0.902784
[36]	train-auc:0.90768
[39]	train-auc:0.915354
[42]	train-auc:0.924582
[45]	train-auc:0.927302
[48]	train-auc:0.933142
[51]	train-auc:0.939583
[54]	train-auc:0.943239
[57]	train-auc:0.946767
[60]	train-auc:0.949225
[63]	train-auc:0.9539
[66]	train-auc:0.957954
[69]	train-auc:0.961568
[72]	train-auc:0.965116
[75]	train-auc:0.968438
[78]	train-auc:0.971091
[81]	train-auc:0.972861
[84]	train-auc:0.974945
[87]	train-auc:0.976455
[90]	train-auc:0.978511
[93]	train-auc:0.979999
[96]	train-auc:0.981519
[99]	train-auc:0.982472
[102]	train-auc:0.983765
[105]	train-auc:0.984513
[108]	train-auc:0.985324
[111]	train-auc:0.986473
[114]	train-auc:0.987452
[117]	trai

'!!! used 959.41 s'

In [1]:
114/0.8

142.5

In [8]:
%%time
predb = clf.predict(d_pred)

CPU times: user 4.25 s, sys: 10 ms, total: 4.26 s
Wall time: 264 ms


In [9]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [10]:
submit.to_csv('xgb2_full.csv',index=False)

___